# **Task 1- Exploratory Data Analysis (EDA)**

### **1. Import Necessary Libraries Required for EDA**
***

In [5]:
# Run "pip install -r requirements.txt" prior to running the eda.ipynb 
# Otherwise, unhash ?and run the following installations below
# pip install summarytools
# pip install python-dotenv

import os
from dotenv import load_dotenv
import urllib.request 
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from summarytools import dfSummary
import seaborn as sns
from scipy import stats
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### **2. Download .db file from URL and Save the Dataset into DataFrame**
***

In [6]:
# Get the database URL from environment variables in .env file
# If .env is not step up but survive.db is downloaded, proceed to step: 
# 'Establishing connection and saving dataset into survive_df'
load_dotenv()
weather_url = os.environ.get("weather_url")
air_quality_url = os.environ.get("air_quality_url")

In [7]:
# Downloading dataset into from given URL and saving into dataset folder
urllib.request.urlretrieve(weather_url, "data/weather.db")
urllib.request.urlretrieve(air_quality_url, "data/air_quality.db")

('data/air_quality.db', <http.client.HTTPMessage at 0x2a47730b3d0>)

In [8]:
# Establishing connection and saving dataset into survive_df
connection = sqlite3.connect("data/weather.db")
weather_df = pd.read_sql_query("SELECT * from weather", connection)
connection.commit()
connection.close()

connection = sqlite3.connect("data/air_quality.db")
air_quality_df = pd.read_sql_query("SELECT * from air_quality", connection)
connection.commit()
connection.close()

# Display the first 5 rows of dataset
weather_df.head()

,data_ref,date,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Min Temperature (deg C),Maximum Temperature (deg C),Min Wind Speed (km/h),Max Wind Speed (km/h),Sunshine Duration (hrs),Cloud Cover (%),Wet Bulb Temperature (deg F),Relative Humidity (%),Air Pressure (hPa),Dew Point Category,Wind Direction,Daily Solar Panel Efficiency
0,4d4d20c2-0167-4bc2-a62c-4df6e11fb2ea,01/04/2014,-,2,2,2,28.4,32.4,7.5,25.9,8.07,18.0,-59.3,100.0,1006,VH,W,High
1,c9d3c825-fd94-4e61-a4bc-2cf0f2208de0,02/04/2014,0,0,0,--,28.8,32.6,--,33.1,7.57,23.0,59.2,100.0,1005,Very High,S,Medium
2,6d445ffb-d4c2-4f94-a27e-224fabe46d10,03/04/2014,0,0,0,0,29.3,--,--,43.2,6.72,16.0,59.7,98.5,1005,Low,E,Medium
3,7a404c50-710f-451a-9a42-a66bb783a3d0,04/04/2014,0,0,0,0,28.6,32.4,7.4,33.1,7.55,20.0,59.1,100.0,1011,Very High,W,Medium
4,0d3194a7-ad1a-456e-9bf8-b1bae730977b,05/04/2014,23.4,15.2,17.2,17.4,26.4,28,5.4,28.1,5.27,74.0,58.9,100.0,1012,Very High,east,Low


In [9]:
#Summary of weather_df
dfSummary(weather_df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,data_ref[object],1. 0895190a-78ae-4138-8d28-b09b152. 6ef41128-cb98-462c-b165-10fe703. 0acf11b1-b4c3-4297-96c2-63af994. 8d9fd485-0436-4c24-8416-447d9f5. cefa5266-ae45-487c-ada3-d72d9e6. 8d6affd7-74b0-4af9-aa7f-f0d1717. c54e8e47-05d0-49d6-9a3b-959a288. aab49f55-a6b2-4595-9629-1400779. 6d40d08d-861d-40f2-b273-f4037a10. d3864f15-a274-4dfe-95cd-672fa011. other,"3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3,487 (99.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADQklEQVR4nO3cwWkbYRRG0TdCJPZGQghUQtKDisg2HauINGGkReJ44yyCS8gdmJxTwb+48BDMp+X9/X2gsFv7Afw/xEZGbGTERkZsZMRGRmxkxEZGbGTERma3LMvXZVme1n4I27ebmS8z83nth7B9zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkdnNzI+Z+b32Q9i+xb8YUXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhklIzYyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsa6iox1FRlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhkls8zM88x8WvshbN7b/nw+fzscDqe1X8K2PR6Pl/3hcDhdr9dfx+Pxde0HsU33+/3pdrud9jMzx+Px9XK5/Fz7UWzasx8IZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZPYzfz8BWfshbNdHX8v5fP7u40n+tcfj8eKzcCpv1lVk/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiPzB0WpV2yec8bJAAAAAElFTkSuQmCC"">",0(0.0%)
2,date[object],1. 17/12/20142. 24/11/20163. 27/04/20154. 09/10/20225. 15/05/20186. 13/12/20177. 09/06/20178. 19/03/20159. 14/10/202210. 26/05/201611. other,"3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)3,487 (99.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADQklEQVR4nO3cwWkbYRRG0TdCJPZGQghUQtKDisg2HauINGGkReJ44yyCS8gdmJxTwb+48BDMp+X9/X2gsFv7Afw/xEZGbGTERkZsZMRGRmxkxEZGbGTERma3LMvXZVme1n4I27ebmS8z83nth7B9zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkdnNzI+Z+b32Q9i+xb8YUXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhklIzYyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsa6iox1FRlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhkls8zM88x8WvshbN7b/nw+fzscDqe1X8K2PR6Pl/3hcDhdr9dfx+Pxde0HsU33+/3pdrud9jMzx+Px9XK5/Fz7UWzasx8IZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZPYzfz8BWfshbNdHX8v5fP7u40n+tcfj8eKzcCpv1lVk/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiPzB0WpV2yec8bJAAAAAElFTkSuQmCC"">",0(0.0%)
3,Daily Rainfall Total (mm)[object],1. 02. -3. --4. 0.25. 0.46. 0.67. 0.88. 1.69. 110. 1.411. other,"1,661 (47.2%)174 (4.9%)167 (4.7%)135 (3.8%)70 (2.0%)65 (1.8%)61 (1.7%)43 (1.2%)38 (1.1%)37 (1.1%)1,066 (30.3%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAEYElEQVR4nO3dQW4TWQBF0Z/IosnEloWUZbAAL6IX24vwAthHFKUGOM0kPYCO6BZxApRvxXDOjNkXulRUpf/IxcPDw4DC5dIH

In [ ]:
#Duplicates of weather data
weather_df['data_ref'].duplicated().value_counts()

In [ ]:
weather_df.duplicated().value_counts()

False    3197
True      320
Name: data_ref, dtype: int64

False    3197
True      320
dtype: int64

In [169]:
weather_df.drop_duplicates(inplace = True)

In [170]:
weather_df.duplicated().value_counts()

False    3197
dtype: int64

In [171]:
#weather_df.to_excel('data/weather.xlsx', index=False)
#air_quality_df.to_excel('data/air_quality.xlsx', index=False)

In [248]:
air_quality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3875 entries, 0 to 3874
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   data_ref      3875 non-null   object
 1   date          3875 non-null   object
 2   pm25_north    3836 non-null   object
 3   pm25_south    3836 non-null   object
 4   pm25_east     3836 non-null   object
 5   pm25_west     3836 non-null   object
 6   pm25_central  3836 non-null   object
 7   psi_north     3836 non-null   object
 8   psi_south     3836 non-null   object
 9   psi_east      3836 non-null   object
 10  psi_west      3836 non-null   object
 11  psi_central   3836 non-null   object
dtypes: object(12)
memory usage: 363.4+ KB


In [259]:
air_quality_df

,data_ref,date,pm25_north,pm25_south,pm25_east,pm25_west,pm25_central,psi_north,psi_south,psi_east,psi_west,psi_central
0,4d4d20c2-0167-4bc2-a62c-4df6e11fb2ea,01/04/2014,20,16,18,23,19,58,56,57,66,59
1,c9d3c825-fd94-4e61-a4bc-2cf0f2208de0,02/04/2014,18,14,-,22,16,57,53,54,61,56
2,6d445ffb-d4c2-4f94-a27e-224fabe46d10,03/04/2014,12,12,10,14,10,55,53,52,59,54
3,7a404c50-710f-451a-9a42-a66bb783a3d0,04/04/2014,--,-,24,19,25,58,57,55,-,56
4,0d3194a7-ad1a-456e-9bf8-b1bae730977b,05/04/2014,18,19,21,13,17,64,64,66,-,66
...,...,...,...,...,...,...,...,...,...,...,...,...
3870,c7ba7dbf-8eb0-42d9-ae26-916cac456dee,09/09/2015,30,30,29,31,29,None,None,None,None,None
3871,85ffffd7-9e90-444e-a030-cbcdc039a3a7,27/04/2016,26,15,15,27,21,None,None,None,None,None
3872,da402a4c-4d07-47d3-8439-411e23e615ef,29/07/2020,--,15,16,12,15,None,None,None,None,None
3873,4c67f80b-faec-44d4-861e-dc5a1e8c5186,29/08/2017,21,-,-,14,18,None,None,None,None,None


In [260]:
air_quality_df = air_quality_df.fillna(0)
air_quality_df = air_quality_df.replace('-', 0)
air_quality_df = air_quality_df.replace('--', 0)

In [261]:
air_quality_df

,data_ref,date,pm25_north,pm25_south,pm25_east,pm25_west,pm25_central,psi_north,psi_south,psi_east,psi_west,psi_central
0,4d4d20c2-0167-4bc2-a62c-4df6e11fb2ea,01/04/2014,20,16,18,23,19,58,56,57,66,59
1,c9d3c825-fd94-4e61-a4bc-2cf0f2208de0,02/04/2014,18,14,0,22,16,57,53,54,61,56
2,6d445ffb-d4c2-4f94-a27e-224fabe46d10,03/04/2014,12,12,10,14,10,55,53,52,59,54
3,7a404c50-710f-451a-9a42-a66bb783a3d0,04/04/2014,0,0,24,19,25,58,57,55,0,56
4,0d3194a7-ad1a-456e-9bf8-b1bae730977b,05/04/2014,18,19,21,13,17,64,64,66,0,66
...,...,...,...,...,...,...,...,...,...,...,...,...
3870,c7ba7dbf-8eb0-42d9-ae26-916cac456dee,09/09/2015,30,30,29,31,29,0,0,0,0,0
3871,85ffffd7-9e90-444e-a030-cbcdc039a3a7,27/04/2016,26,15,15,27,21,0,0,0,0,0
3872,da402a4c-4d07-47d3-8439-411e23e615ef,29/07/2020,0,15,16,12,15,0,0,0,0,0
3873,4c67f80b-faec-44d4-861e-dc5a1e8c5186,29/08/2017,21,0,0,14,18,0,0,0,0,0


In [262]:
air_quality_df = air_quality_df.astype({col: float for col in air_quality_df.columns[2:]})


In [263]:
air_quality_df.duplicated().value_counts()

False    3236
True      639
dtype: int64

In [264]:
air_quality_df['data_ref'].duplicated().value_counts()

False    3197
True      678
Name: data_ref, dtype: int64

In [265]:
air_quality_df['overall_dup']= air_quality_df.duplicated()
air_quality_df['data_ref_dup']= air_quality_df['data_ref'].duplicated()

In [266]:
dataframe_issues = air_quality_df[(air_quality_df['data_ref_dup'] == True) & (air_quality_df['overall_dup'] == False)]
issues = dataframe_issues.sort_values(by= 'data_ref')
air_quality_df[air_quality_df['data_ref'].isin(issues['data_ref'])].sort_values(by= 'data_ref')
weird_dups = air_quality_df[air_quality_df['data_ref'].isin(issues['data_ref'])].sort_values(by= 'data_ref')

'''# Identify duplicated rows and duplicated 'data_ref' entries
air_quality_df['overall_dup'] = air_quality_df.duplicated()
air_quality_df['data_ref_dup'] = air_quality_df['data_ref'].duplicated()

# Select rows where 'data_ref' is duplicated but the overall row is not duplicated
dataframe_issues = air_quality_df[(air_quality_df['data_ref_dup']) & (~air_quality_df['overall_dup'])]

# Sort the identified issues by 'data_ref' for better readability
issues_sorted = dataframe_issues.sort_values(by='data_ref')

# Filter the original dataframe for rows that have 'data_ref' in the identified issues
weird_dups = air_quality_df[air_quality_df['data_ref'].isin(issues_sorted['data_ref'])].sort_values(by='data_ref')

# Display the results
print(weird_dups)'''

,data_ref,date,pm25_north,pm25_south,pm25_east,pm25_west,pm25_central,psi_north,psi_south,psi_east,psi_west,psi_central,overall_dup,data_ref_dup
3860,02b1b301-119b-439c-9f7f-19d7f3d6406d,26/10/2017,19.0,20.0,25.0,18.0,27.0,0.0,0.0,0.0,0.0,0.0,False,True
3821,02b1b301-119b-439c-9f7f-19d7f3d6406d,26/10/2017,0.0,0.0,0.0,0.0,0.0,60.0,63.0,63.0,62.0,65.0,False,False
3857,0405e809-4714-4b38-a18e-4de8d04a4a6a,28/11/2020,0.0,13.0,9.0,5.0,10.0,0.0,0.0,0.0,0.0,0.0,False,True
3818,0405e809-4714-4b38-a18e-4de8d04a4a6a,28/11/2020,0.0,0.0,0.0,0.0,0.0,0.0,48.0,37.0,29.0,36.0,False,False
3846,0487564a-4c04-469e-b345-8f70bb2359af,01/12/2019,7.0,9.0,9.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3827,e5701968-933b-4fbb-9073-776084101bd6,10/07/2016,0.0,0.0,0.0,0.0,0.0,0.0,38.0,37.0,36.0,35.0,False,False
3816,e8ccd011-1612-43a8-9ea5-80d883fb88a2,27/07/2014,0.0,0.0,0.0,0.0,0.0,55.0,52.0,54.0,56.0,0.0,False,False
3855,e8ccd011-1612-43a8-9ea5-80d883fb88a2,27/07/2014,15.0,12.0,14.0,17.0,15.0,0.0,0.0,0.0,0.0,0.0,False,True
3802,f420775d-da32-41c6-b13d-62259d89ef92,01/03/2022,0.0,0.0,0.0,0.0,0.0,51.0,46.0,53.0,35.0,0.0,False,False


In [273]:
combined_air_quality_df= air_quality_df.groupby('data_ref').sum()

C:\Users\Hello\AppData\Local\Temp\ipykernel_20476\157037687.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  combined_air_quality_df= air_quality_df.groupby('data_ref').sum()


In [282]:
combined_air_quality_df.drop(['overall_dup', 'data_ref_dup'], axis =1 )

,pm25_north,pm25_south,pm25_east,pm25_west,pm25_central,psi_north,psi_south,psi_east,psi_west,psi_central
data_ref,,,,,,,,,,
002fbd73-5c3c-46e3-a294-bf62b9933d1a,22.0,0.0,20.0,25.0,21.0,66.0,66.0,66.0,72.0,0.0
00336736-2407-4fa9-841d-d5432e9e32d4,10.0,0.0,0.0,11.0,11.0,0.0,38.0,36.0,39.0,35.0
00352582-da85-4794-964e-1d35b06d3fa3,23.0,25.0,22.0,0.0,19.0,62.0,67.0,61.0,57.0,58.0
0045a78b-254a-4291-a23c-d3a7afd33a7c,9.0,0.0,11.0,0.0,9.0,47.0,39.0,52.0,40.0,52.0
0069491d-beae-421a-ae43-126582453650,9.0,6.0,9.0,8.0,9.0,44.0,48.0,45.0,34.0,45.0
...,...,...,...,...,...,...,...,...,...,...
ffcb7f51-63a2-47c4-b97f-24b830b40b38,16.0,22.0,17.0,16.0,16.0,56.0,61.0,57.0,55.0,57.0
ffd2b9e1-c66e-4f5c-9b8e-e6261de2de13,9.0,13.0,13.0,6.0,8.0,44.0,54.0,53.0,0.0,51.0
ffd5e119-2b08-4bef-af7b-031659dbb3a8,20.0,25.0,23.0,16.0,21.0,58.0,61.0,61.0,56.0,58.0


In [285]:
combined_air_quality_df = combined_air_quality_df.reset_index()

In [287]:
set(combined_air_quality_df['data_ref']) == set(weather_df['data_ref'])

True

In [289]:
air_weather_df = pd.merge(weather_df, combined_air_quality_df, on= 'data_ref', how= 'left')
air_weather_df

,data_ref,date,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Min Temperature (deg C),Maximum Temperature (deg C),Min Wind Speed (km/h),Max Wind Speed (km/h),...,pm25_east,pm25_west,pm25_central,psi_north,psi_south,psi_east,psi_west,psi_central,overall_dup,data_ref_dup
0,4d4d20c2-0167-4bc2-a62c-4df6e11fb2ea,01/04/2014,-,2,2,2,28.4,32.4,7.5,25.9,...,18.0,23.0,19.0,58.0,56.0,57.0,66.0,59.0,0,0
1,c9d3c825-fd94-4e61-a4bc-2cf0f2208de0,02/04/2014,0,0,0,--,28.8,32.6,--,33.1,...,0.0,22.0,16.0,57.0,53.0,54.0,61.0,56.0,0,0
2,6d445ffb-d4c2-4f94-a27e-224fabe46d10,03/04/2014,0,0,0,0,29.3,--,--,43.2,...,10.0,14.0,10.0,55.0,53.0,52.0,59.0,54.0,0,0
3,7a404c50-710f-451a-9a42-a66bb783a3d0,04/04/2014,0,0,0,0,28.6,32.4,7.4,33.1,...,24.0,19.0,25.0,58.0,57.0,55.0,0.0,56.0,0,0
4,0d3194a7-ad1a-456e-9bf8-b1bae730977b,05/04/2014,23.4,15.2,17.2,17.4,26.4,28,5.4,28.1,...,21.0,13.0,17.0,64.0,64.0,66.0,0.0,66.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3512,cf6a7375-6049-4098-9612-43dcbef0b343,21/11/2015,0,0,0,0,27.6,31.7,6.2,28.8,...,22.0,16.0,26.0,78.0,86.0,102.0,64.0,84.0,1,1
3513,cdc5b607-ee26-4a9c-a4ac-bd4f8b9f18bc,10/06/2020,13.4,11,12.6,13.4,29.4,-,14.4,50,...,8.0,5.0,8.0,38.0,39.0,32.0,27.0,37.0,0,0
3514,9f9533f5-41c1-4b5a-9b3f-a3f57ee3647a,27/10/2021,3.8,--,3.8,3.8,29.1,33.1,7.3,29.6,...,10.0,5.0,10.0,55.0,36.0,53.0,54.0,59.0,0,0
3515,07afbfbb-3e93-467e-afa6-8efd4aaca1e0,03/06/2021,1.2,1.2,1.2,1.2,29.6,32.5,11.1,-,...,0.0,10.0,12.0,55.0,41.0,43.0,48.0,54.0,0,0


In [290]:
air_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3517 entries, 0 to 3516
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   data_ref                       3517 non-null   object 
 1   date                           3517 non-null   object 
 2   Daily Rainfall Total (mm)      3517 non-null   object 
 3   Highest 30 Min Rainfall (mm)   3517 non-null   object 
 4   Highest 60 Min Rainfall (mm)   3517 non-null   object 
 5   Highest 120 Min Rainfall (mm)  3517 non-null   object 
 6   Min Temperature (deg C)        3517 non-null   object 
 7   Maximum Temperature (deg C)    3517 non-null   object 
 8   Min Wind Speed (km/h)          3517 non-null   object 
 9   Max Wind Speed (km/h)          3502 non-null   object 
 10  Sunshine Duration (hrs)        3501 non-null   float64
 11  Cloud Cover (%)                3501 non-null   float64
 12  Wet Bulb Temperature (deg F)   3517 non-null   f

In [297]:
air_weather_df.to_excel('data/combined.xlsx', index=False)